# Multiobjective Vehicle Routing Problems With Simultaneous Delivery and Pickup and Time Windows 

In [163]:
import random
import math
import copy
import numpy as np
import matplotlib.pyplot as plt

In [164]:
#  Distance of a specific route.
def node_distance(A,B):
    return math.sqrt((A.x-B.x)**2 + (A.y-B.y)**2)
#  Calculate the distance matrix of all nodes
def distance_matrix(Nodes):
    n = len(Nodes)
    matrix = [[0 for i in range(n)] for j in range(n)]
    for i in range(n):
        for j in range(n):
            matrix[i][j] = node_distance(Nodes[i],Nodes[j])
    return matrix

1. Define Node, Vehicle and RoutePlan

In [165]:
class Node():
    def __init__(self,id,x,y,delivery,pickup,T_l,T_r,service_time):
        self.id = id
        self.x = x
        self.y = y
        self.delivery = delivery
        self.pickup = pickup
        self.loadchange = max(0,delivery-pickup)
        self.T_l = T_l
        self.T_r = T_r
        self.service_time = service_time

class Vehicle():
    def __init__(self,depot,depotload,capacity,speed):
        self.depot = depot
        self.load = []
        self.depotload = depotload
        self.capacity = capacity
        self.speed = speed
        self.route = []
        self.arrival_time = [] # The time that the vehicle arrives at each node
        self.departure_time = []

    def current_node(self):
        if self.route == []:
            return self.depot
        return self.route[-1]
    
    def add_node(self,node):
        if self.route == []:
            arrivaltime = node.T_l
            departuretime = arrivaltime + node.service_time
            load = node.loadchange + self.depotload
        else:
            arrivaltime = self.departure_time[-1] + node_distance(self.current_node(),node)/self.speed
            departuretime = arrivaltime + node.service_time
            load = max(0,node.pickup - node.delivery) + self.load[-1]
        self.arrival_time.append(arrivaltime)
        self.departure_time.append(departuretime)
        self.load.append(load)
        self.route.append(node)

    def route_length(self):
        length = 0
        for i in range(len(self.route)-1):
            length += node_distance(self.route[i],self.route[i+1])
        return length
    
    def update(self):
        self.load = [self.depotload + self.route[0].loadchange]
        self.arrival_time = [self.route[0].T_l]
        self.departure_time = [self.route[0].T_l + self.route[0].service_time]
        for i,node in enumerate(self.route):
            if i == 0:
                continue
            self.load.append(max(0,node.delivery - node.pickup) + self.load[-1])
            arrivaltime = self.departure_time[-1] + node_distance(self.route[i-1],node)/self.speed
            self.arrival_time.append(arrivaltime)
            self.departure_time.append(arrivaltime + node.service_time)
        return
    
class RoutePlan():
    def __init__(self,depot,capacity):
        self.depot = depot
        self.capacity = capacity
        self.vehicles = []
        self.objectives = np.array([self.f1(),self.f2(),self.f3(),self.f4(),self.f5()])
        self.rank = None
        self.crowding_distance = None
        self.domination_count = 0
        self.dominated_solutions = []

    def add_vehicle(self,vehicle):
        self.vehicles.append(vehicle)

    def f1(self): # number of vehicles
        return len(self.vehicles)
    
    def f2(self): # Total Travel Distance
        total_distance = 0
        for vehicle in self.vehicles:
            total_distance += vehicle.route_length()
        return total_distance
    
    def f3(self): # Travel time of the longest route
        max_time = 0
        for vehicle in self.vehicles:
            if vehicle.time[-1] > max_time:
                max_time = vehicle.time[-1]
        return max_time
    
    def f4(self): # Total waiting time due to early arrival
        waiting_time = 0
        for vehicle in self.vehicles:
            for i,node in enumerate(vehicle.route):
                if vehicle.arrival_time[i] < node.T_l:
                    waiting_time += node.T_l - vehicle.arrival_time[i]
        return waiting_time
    
    def f5(self): # Total delay time due to late arrival
        delay_time = 0
        for vehicle in self.vehicles:
            for i,node in enumerate(vehicle.route):
                if vehicle.arrival_time[i] > node.T_r:
                    delay_time += vehicle.arrival_time[i] - node.T_r
        return delay_time
    
    def dominate(self,RP):
        compare = self.F() > RP.F()
        if compare.all() == True:
            return True
        return False

2. Generate Nodes and Initial Solution

In [166]:
def Generate_Nodes(N):
    # 1. Generate N nodes with random x,y coordinates and demand
    Nodes = []
    location = list(set((random.randrange(800), random.randrange(400))for _ in range(N)))
    PD = list((random.randrange(10,40),random.randrange(30,50))for _ in range(N))
    # 2. Generate N time windows
    Time_windows = []
    T_left = []
    T_right = []
    service_times = [random.randint(10,60) for _ in range(N)]

    for i in range(N):
        l_hour, l_minute = random.randint(9,17), random.randint(0,59)
        r_hour, r_minute = l_hour + 2, random.randint(0,59)
        l_time= f"{str(l_hour).zfill(2)}:{str(l_minute).zfill(2)}"
        r_time= f"{str(r_hour).zfill(2)}:{str(r_minute).zfill(2)}"
        Time_windows.append((l_time,r_time))
        T_left.append(int(l_time.split(':')[0])*60 + int(l_time.split(':')[1]))
        T_right.append(int(r_time.split(':')[0])*60 + int(r_time.split(':')[1]))
    
    for i in range(N):
        node = Node(i+1,location[i][0],location[i][1],PD[i][0],PD[i][1],T_left[i],T_right[i],service_times[i])
        Nodes.append(node)
    return Nodes

def Initial_Solution(Nodes,capacity,speed,max_delay,N):
    population = []
    for i in range(N):
        RP = RoutePlan(Nodes[0],capacity)
        unvisited = list(range(1,len(Nodes)))

        vehicle = Vehicle(Nodes[0],50,200,speed)
        while unvisited:
            current_node = vehicle.current_node()
        # Find the next node that satisfies the capacity and time windows constraints
            feasible_nodes = []
            load = vehicle.load[-1] if vehicle.load else vehicle.depotload
            departure_time = vehicle.departure_time[-1] if vehicle.departure_time else 0
            for i in unvisited:
                if load + Nodes[i].loadchange <= vehicle.capacity:
                    if departure_time + node_distance(current_node,Nodes[i])/vehicle.speed <= Nodes[i].T_r + max_delay:
                        feasible_nodes.append(i)
            if feasible_nodes:
                if len(vehicle.route) == 0:
                    next_node = random.choice(feasible_nodes)
                else:
                    next_node = min(feasible_nodes,key=lambda x:node_distance(current_node,Nodes[x]))
                vehicle.add_node(Nodes[next_node])
                unvisited.remove(next_node)
            else:
                RP.add_vehicle(vehicle)
                vehicle = Vehicle(Nodes[0],50,200,speed)
        RP.add_vehicle(vehicle)
    population.append(RP)
    return population

3. Non-dominate sorting and Pareto Front

In [167]:
def fast_non_dominated_sort(population):
    fronts = [[]]
    for RP in population:
        for other_RP in population:
            if RP.dominate(other_RP):
                RP.dominated_solutions.append(other_RP)
            elif other_RP.dominate(RP):
                RP.domination_count += 1
        if RP.domination_count == 0:
            RP.rank = 0
            fronts[0].append(RP)

    for i in range(len(fronts)):
        next_front = []
        for RP in fronts[i]:
            for other_RP in fronts[i]:
                other_RP.domination_count -= 1
                if other_RP.domination_count == 0:
                    other_RP.rank = i+1
                    next_front.append(other_RP)
        fronts.append(next_front)
    return fronts

def crowding_distance(front): # single front
    if len(front) > 0:
        for RP in front:
            RP.crowding_distance = 0
        for i in range(len(front[0].objectives)):
            front.sort(key = lambda RP: RP.objectives[i])
            front[0].crowding_distance = front[-1].crowding_distance = float('inf')
            m_values = [RP.objectives[i] for RP in front]
            scale = max(m_values) - min(m_values)
            if scale == 0: # in case all the values are the same
                scale = 1
            for j in range(1,len(front)-1):
                front[j].crowding_distance += (m_values[j+1] - m_values[j-1])/scale  ###########
    return

def order(RP1,RP2): #return better RP between two
    if RP1.rank < RP2.rank or \
    (RP1.rank == RP2.rank) and (RP1.crowding_distance > RP2.crowding_distance):
        return RP1
    return RP2

4. Perturbation Funtion

In [168]:
def insertion(RP,time_matrix,max_delay):
    RP = copy.deepcopy(RP)
    # 1.  Select a node to remove
    from_vehicle = min(RP.vehicles, key = lambda x : len(x.route)) # vehicle with the least nodes
    i = random.randint(0,len(from_vehicle.route)-1)
    departure_time = from_vehicle.departure_time[i]
    node = from_vehicle.route.pop(i)

    # 2. Find the set of nodes that can be inserted to
    to_node_set = []
    for vehicle in [vehicle for vehicle in RP.vehicles if vehicle is not from_vehicle]: #Do not insert to the same vehicle
        if vehicle.load[-1] + node.loadchange > vehicle.capacity: # capacity constraint
            continue
        for n in vehicle.route:
            if departure_time + time_matrix[node.id][n.id] <= n.T_r + max_delay: # time window constraint
                to_node_set.append(n)

    # If there is no feasible node to insert to, insert it back to the original vehicle
    if not to_node_set:
        from_vehicle.route.insert(i,node)
        return RP
    
    from_vehicle.update()
    # 3. Insert the removed node to a feasible position
    to_node = random.choice(to_node_set)
    for vehicle in RP.vehicles:
        if to_node.id in [node.id for node in vehicle.route]:
            to_vehicle = vehicle
            break
    j = to_vehicle.route.index(to_node.id)
    to_vehicle.route.insert(j,node)

    # 4.Update RoutePlan
    to_vehicle.update()
    return RP

def two_optswap(Routeplan,Nodes):
    pass

# Select some nodes with approximate time windows from multiple vehicles and insert them into best position
def LNS(RP,Nodes,max_delay):

    node_i = random.choice(Nodes)
    T_l, T_r = node_i.T_l, node_i.T_r
    removed_nodes = []
    for node in Nodes:
        if node.T_l >= T_l and node.T_r <= T_r + max_delay :
            removed_nodes.append(node)
    
    if len(removed_nodes) == 1:
        return RP
    
    # Remove nodes from vehicle routes
    for vehicle in RP.vehicles:
        for node in vehicle.route:
            if node in removed_nodes:
                vehicle.route.remove(node)
                removed_nodes.remove(node)
        vehicle.update()
    
    # Insert nodes to the best position
    return

4. Main Function

In [169]:
# 1. Generate N nodes, a depot and the initial solution
depot = Node(0,400,200,0,0,0,1440,0)
N = 20
speed = 1
max_delay = 20
customers = Generate_Nodes(N)
Nodes = [depot] + customers
distance_matrix = distance_matrix(Nodes)
time_matrix = (np.array(distance_matrix) / speed).tolist()
pop_size = 20
population = Initial_Solution(Nodes,200,speed,max_delay,20)
max_iter = 5
# 2. Fast non-dominated sort
for iter in range(max_iter):
    for RP in population:
        new_RP = insertion(RP,time_matrix,max_delay)
        population.append(new_RP)
    
    fronts = fast_non_dominated_sort(population)
    for front in fronts:
        crowding_distance(front)
        # Generate new population
    new_population = []
    for front in fronts:
        if len(new_population) + len(front) > pop_size:
            break
        new_population.append(front)

KeyboardInterrupt: 

References:  
[1]	J. Wang, Y. Zhou, Y. Wang, J. Zhang, C. L. P. Chen, and Z. Zheng, "Multiobjective Vehicle Routing Problems With Simultaneous Delivery and Pickup and Time Windows: Formulation, Instances, and Algorithms | IEEE Journals & Magazine | IEEE Xplore," IEEE Transactions on Cybernetics, vol. 46, no. 3, 2016, doi: 10.1109/TCYB.2015.2409837.  
[2]	C. Prins, "A simple and effective evolutionary algorithm for the vehicle routing problem," (in English), Comput. Oper. Res., vol. 31, no. 12, pp. 1985-2002, Oct 2004, doi: 10.1016/s0305-0548(03)00158-8.  
[3]	https://github.com/baopng/NSGA-II/tree/master